# Bronze Layer EDA - Local Version

This notebook performs exploratory analysis of GitHub Archive data locally using pandas.
Use this for development and testing before deploying to Databricks.

## Requirements
```bash
pip install pandas requests matplotlib
```

## 1. Setup & Data Download

In [ ]:
import gzip
import json
from datetime import date, datetime

import matplotlib.pyplot as plt
import pandas as pd
import requests

# Configuration
SAMPLE_DATE = date(2024, 1, 15)  # Adjust to a recent date with available data
SAMPLE_HOUR = 10  # Single hour for quick analysis

print(f"Sample date: {SAMPLE_DATE}")
print(f"Sample hour: {SAMPLE_HOUR}")

In [ ]:
def download_gharchive_hour(archive_date: date, hour: int) -> list[dict]:
    """Download and parse one hour of GH Archive data."""
    url = f"https://data.gharchive.org/{archive_date.isoformat()}-{hour}.json.gz"
    print(f"Downloading: {url}")
    
    response = requests.get(url, timeout=120)
    response.raise_for_status()
    
    decompressed = gzip.decompress(response.content).decode('utf-8')
    events = []
    
    for line in decompressed.strip().split('\n'):
        if line:
            try:
                events.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    
    print(f"Parsed {len(events):,} events")
    return events

In [ ]:
# Download sample data
events = download_gharchive_hour(SAMPLE_DATE, SAMPLE_HOUR)
print(f"\nTotal events: {len(events):,}")

## 2. Create DataFrame

In [ ]:
# Extract key fields into a DataFrame
records = []
for event in events:
    records.append({
        'event_id': event.get('id'),
        'event_type': event.get('type'),
        'actor_id': event.get('actor', {}).get('id'),
        'actor_login': event.get('actor', {}).get('login'),
        'repo_id': event.get('repo', {}).get('id'),
        'repo_name': event.get('repo', {}).get('name'),
        'created_at': event.get('created_at'),
        'public': event.get('public'),
        'payload_action': event.get('payload', {}).get('action'),
    })

df = pd.DataFrame(records)
df['created_at'] = pd.to_datetime(df['created_at'])

print(f"DataFrame shape: {df.shape}")
df.head()

In [ ]:
# DataFrame info
df.info()

## 3. Event Type Analysis

In [ ]:
# Event type distribution
event_counts = df['event_type'].value_counts()
event_pct = df['event_type'].value_counts(normalize=True) * 100

event_summary = pd.DataFrame({
    'count': event_counts,
    'percentage': event_pct.round(2)
})

print("Event Type Distribution:")
event_summary

In [ ]:
# Visualize event types
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart
event_counts.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Count')
axes[0].set_ylabel('Event Type')
axes[0].set_title('GitHub Event Type Distribution')
axes[0].invert_yaxis()

# Pie chart (top 5)
top_5 = event_counts.head(5)
other = event_counts[5:].sum()
pie_data = pd.concat([top_5, pd.Series({'Other': other})])
pie_data.plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
axes[1].set_ylabel('')
axes[1].set_title('Top 5 Event Types')

plt.tight_layout()
plt.show()

In [ ]:
# Event categories
def categorize_event(event_type):
    categories = {
        'Code Activity': ['PushEvent', 'CreateEvent', 'DeleteEvent', 'CommitCommentEvent'],
        'Pull Requests': ['PullRequestEvent', 'PullRequestReviewEvent', 'PullRequestReviewCommentEvent'],
        'Issues': ['IssuesEvent', 'IssueCommentEvent'],
        'Social': ['WatchEvent', 'ForkEvent'],
        'Releases': ['ReleaseEvent'],
    }
    for cat, types in categories.items():
        if event_type in types:
            return cat
    return 'Other'

df['category'] = df['event_type'].apply(categorize_event)
category_counts = df['category'].value_counts()

print("\nEvent Categories:")
category_counts

## 4. Actor Analysis

In [ ]:
# Top actors by event count
top_actors = df.groupby('actor_login').agg(
    event_count=('event_id', 'count'),
    event_types=('event_type', 'nunique'),
    repos=('repo_name', 'nunique')
).sort_values('event_count', ascending=False).head(20)

print("Top 20 Most Active Users:")
top_actors

In [ ]:
# Actor activity distribution
actor_activity = df.groupby('actor_login').size()

print(f"\nActor Activity Statistics:")
print(f"  Total unique actors: {len(actor_activity):,}")
print(f"  Mean events per actor: {actor_activity.mean():.2f}")
print(f"  Median events per actor: {actor_activity.median():.0f}")
print(f"  Max events by one actor: {actor_activity.max():,}")

## 5. Repository Analysis

In [ ]:
# Top repositories by activity
top_repos = df.groupby('repo_name').agg(
    event_count=('event_id', 'count'),
    event_types=('event_type', 'nunique'),
    unique_actors=('actor_login', 'nunique')
).sort_values('event_count', ascending=False).head(20)

print("Top 20 Most Active Repositories:")
top_repos

In [ ]:
# Repos receiving stars (WatchEvents)
starred_repos = df[df['event_type'] == 'WatchEvent'].groupby('repo_name').size().sort_values(ascending=False).head(20)

print("\nMost Starred Repos (in this sample):")
starred_repos

## 6. Temporal Analysis

In [ ]:
# Events by minute
df['minute'] = df['created_at'].dt.minute
events_by_minute = df.groupby('minute').size()

plt.figure(figsize=(12, 4))
events_by_minute.plot(kind='line', marker='o')
plt.xlabel('Minute of Hour')
plt.ylabel('Event Count')
plt.title(f'Events by Minute (Hour {SAMPLE_HOUR})')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Data Quality Checks

In [ ]:
# Null analysis
null_counts = df.isnull().sum()
null_pct = (df.isnull().sum() / len(df) * 100).round(2)

null_summary = pd.DataFrame({
    'null_count': null_counts,
    'null_pct': null_pct
})

print("Null Analysis:")
null_summary

In [ ]:
# Duplicate event IDs
duplicates = df['event_id'].duplicated().sum()
print(f"Duplicate event IDs: {duplicates}")
print(f"Duplicate rate: {duplicates / len(df) * 100:.2f}%")

## 8. Sample Event Deep Dive

In [ ]:
# Sample raw event (PushEvent)
push_events = [e for e in events if e.get('type') == 'PushEvent']
if push_events:
    print("Sample PushEvent:")
    print(json.dumps(push_events[0], indent=2))

In [ ]:
# PushEvent commit analysis
push_commits = []
for e in push_events:
    push_commits.append({
        'repo': e['repo']['name'],
        'actor': e['actor']['login'],
        'commits': e['payload'].get('size', 0),
        'branch': e['payload'].get('ref', '').replace('refs/heads/', '')
    })

push_df = pd.DataFrame(push_commits)
print(f"\nPushEvent Statistics:")
print(f"  Total pushes: {len(push_df):,}")
print(f"  Total commits: {push_df['commits'].sum():,}")
print(f"  Mean commits per push: {push_df['commits'].mean():.2f}")
print(f"\nCommits per push distribution:")
push_df['commits'].describe()

## 9. Summary

In [ ]:
print("="*60)
print("EDA SUMMARY")
print("="*60)
print(f"\nData Source: GH Archive {SAMPLE_DATE} Hour {SAMPLE_HOUR}")
print(f"\nVolume:")
print(f"  Total events: {len(df):,}")
print(f"  Unique event types: {df['event_type'].nunique()}")
print(f"  Unique actors: {df['actor_login'].nunique():,}")
print(f"  Unique repos: {df['repo_name'].nunique():,}")
print(f"\nTop Event Types:")
for event_type, count in event_counts.head(5).items():
    print(f"  {event_type}: {count:,} ({count/len(df)*100:.1f}%)")
print(f"\nData Quality:")
print(f"  Duplicate rate: {duplicates / len(df) * 100:.2f}%")
print(f"  Null actor_login: {df['actor_login'].isnull().sum()}")